# Ejercicio 1

In [2]:
import pandas as pd

class NaiveBayesClassifier:

    def train(self, df, alpha=1):
        grouped_classes = df.groupby('Nacionalidad')
        self.classes = df['Nacionalidad'].unique()
        self.attributes = df.columns[0: -1]
        self.class_probabilities = {}
        self.attribute_probabilities = {}
        total_len = len(df)

        for _class, grouped in grouped_classes:
            data_len = len(grouped)
            self.class_probabilities[_class] = (data_len + alpha) / (total_len + alpha * len(self.classes))
            self.attribute_probabilities[_class] = {}
            for attribute in df.columns[0:-1]:    # Skipping last column (Nacionalidad)
                self.attribute_probabilities[_class][attribute] = (grouped[attribute].sum() + alpha) / (data_len + alpha * len(self.attributes))
    
    def print_probabilities(self, values):
        prediction = {}
        for _class in self.classes:
            probability = self.class_probabilities[_class]
            for i,value in enumerate(values):
                if value == 1:
                    probability *= self.attribute_probabilities[_class][self.attributes[i]]
                else: 
                    probability *= 1 - self.attribute_probabilities[_class][self.attributes[i]] # 1 - P(A) = P(not A)
            prediction[_class] = probability
        for _class, value in prediction.items():
            print(_class, value / sum(prediction.values()))  # divide the probability by the sum of all probabilities to get the normalized probability


file = 'inputs/PreferenciasBritanicos.xlsx'
brits_df = pd.read_excel(file)
classifier = NaiveBayesClassifier()
classifier.train(brits_df)
# 1.B
classifier.print_probabilities([1, 0, 1, 1, 1])
print()
# 1.C
classifier.print_probabilities([0, 1, 1, 0, 1])

I 0.25260461382104393
E 0.7473953861789561

I 0.623508361311908
E 0.37649163868809205


# Ejercicio 2

## Paso 1: preprocesamiento del dataset
Primero necesitamos hacer un preprocesamiento de los datos. Para esto decidimos reducir el dataset a 4 categorías, preferencialmente que no guarden demasiada correlación entre ellas en cuanto a las palabras utilizadas y asegurarnos que el modelo logre diferenciarlas correctamente sin tener que lidiar con las otras categorías. Una vez que el objetivo sea cumplido agregaremos devuelta las categorías filtradas para utilizar el dataset completo.

Para el preprocesamiento de los titulos utilizaremos tecnicas utilizadas en NLP y las aplicaremos a los titulos, procedemos a la tokenización de las palabras en los titulos,  

In [8]:
import re
import nltk
import pandas as pd
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

class NewsClassifier:

    def __init__(self):
        self.train_df = None
        self.test_df = None
        self.vocabulary = None
        self.vocabulary_size = None
        self.word_freq_per_class = defaultdict(lambda: defaultdict(lambda: 0))
        self.total_tokens_per_class = defaultdict(int)
        self.class_frequency_map = None
        self.labels = None
        self.confusion_matrix = None

    def preprocess_text_spanish(self,text, use_stopwords, use_stemmer):
        # Convert to lowercase and remove punctuation
        text = re.sub(r'[^\w\s]', '', text.lower())
    
        # Tokenization
        tokens = text.split()
        if use_stopwords:
            # Remove stop words
            stop_words = set(stopwords.words("spanish"))
            tokens = [token for token in tokens if token not in stop_words]
    
        if use_stemmer:
            # Stemming (use SnowballStemmer for Spanish)
            stemmer = SnowballStemmer("spanish")
            tokens = [stemmer.stem(token) for token in tokens]
        return tokens

    def split_dataset(self, test_percentage, dataset_path, categories):
        file = dataset_path
        news_df = pd.read_excel(file)
        accepted_cats = categories
        filtered_df = news_df[news_df['categoria'].isin(accepted_cats)]
        if test_percentage != 0:
            self.train_df, self.test_df = train_test_split(filtered_df, test_size=test_percentage/100, random_state=42)
            self.labels = self.train_df['categoria']
        else:
            self.train_df = filtered_df
            self.labels = filtered_df['categoria']

    def train(self):
        # a cada titular le aplico la función de tokenización
        preprocessed_titles = [self.preprocess_text_spanish(title, True, True) for title in self.train_df['titular']]
        preprocessed_titles_strings = [" ".join(tokens) for tokens in preprocessed_titles]
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(preprocessed_titles_strings)

        # Build the vocabulary
        self.vocabulary = vectorizer.get_feature_names_out()
        self.vocabulary_size = len(self.vocabulary)

        self.class_frequency_map = self.labels.value_counts(normalize=True).to_dict() #calcula la frecuencia relativa para cada clase

        # Count word occurrences in each class
        for i, doc in enumerate(X):
            tokens = preprocessed_titles[i]
            self.total_tokens_per_class[self.labels.iloc[i]] += len(tokens)
            for word_idx in doc.indices:
                word = self.vocabulary[word_idx]
                for token in tokens:
                    self.word_freq_per_class[token][self.labels.iloc[i]] += 1
        
                
    def predict_title(self, new_title):

        # Preprocess the new title
        new_title_tokens = self.preprocess_text_spanish(new_title, False, False)

        # Calculate conditional probabilities
        predicted_probabilities = {}
        for class_label in self.class_frequency_map.keys():
            class_frequency = self.class_frequency_map[class_label]
            prob_word_given_class = class_frequency
            for token in new_title_tokens:
                word_freq = self.word_freq_per_class[token][class_label]
                prob_word_given_class *= (word_freq+1) / (sum(self.word_freq_per_class[token].values()) + len(self.class_frequency_map.keys()))
            predicted_probabilities[class_label] =prob_word_given_class

        # Predict the category with the highest probability
        #print(predicted_probabilities)
        predicted_category = max(predicted_probabilities, key=predicted_probabilities.get)
        #print("Predicted Category:", predicted_category)
        return predicted_category

    def calculate_confusion_matrix_ROC(self,label):
        roc_matrix = {}
        roc_matrix[label] = {}
        roc_matrix['other'] = {}
        roc_matrix[label][label] =0
        roc_matrix[label]['other'] =0
        roc_matrix['other'][label] =0
        roc_matrix['other']['other'] =0
        for index, row in self.test_df.iterrows():
            expected_result = row['categoria']
            predicted_result = self.predict_title(row['titular'])
            if expected_result==predicted_result:
                if expected_result == label:
                    roc_matrix[label][label] += 1
                else:
                   roc_matrix['other']['other'] += 1 
            else:
                if expected_result == label:
                    roc_matrix[label]['other'] += 1
                else:
                    roc_matrix['other'][label] += 1
        return roc_matrix
    
    
    def calculate_confusion_matrix(self):
        label_ocurrences = {}
        self.confusion_matrix = {}
        for label in self.class_frequency_map.keys():
            label_ocurrences[label] = (self.test_df['categoria'] == label).sum()
            self.confusion_matrix[label] = {}
            for column_label in self.class_frequency_map.keys():
                self.confusion_matrix[label][column_label] = 0
        for index, row in self.test_df.iterrows():
            expected_result = row['categoria']
            predicted_result = self.predict_title(row['titular'])
            self.confusion_matrix[expected_result][predicted_result] += 1
    
    def change_confusion_matrix_to_freq(self):
        for row in self.confusion_matrix.keys():
            for column in self.confusion_matrix[row].keys():
                self.confusion_matrix[row][column] /= label_ocurrences[row]
                
    def print_confusion_matrix(self):
        for row in self.confusion_matrix.keys():
            for column in self.confusion_matrix[row].keys():
                print(row, column, self.confusion_matrix[row][column])
        for label in label_ocurrences.keys():
            print(label,":", label_ocurrences[label])
            
    def calculate_accuracy(self, matrix):
        asserted = 0
        total = 0
        for row in matrix.keys():
            for column in matrix.keys():
                if row == column:
                    asserted += matrix[row][column]
                total += matrix[row][column]
        return asserted / total
        
    def calculate_true_positives(self, category, matrix):
        false_negatives = 0
        true_positives = 0
        for column in matrix.keys():
            if column != category:
                false_negatives += matrix[category][column]
            else:
                true_positives += matrix[category][column]             
        return true_positives/(true_positives + false_negatives)
        
    def calculate_precision(self, category, matrix):
        true_positives = 0
        false_positives = 0
        for row in matrix.keys():
            if row != category:
                false_positives += matrix[row][category]
            else:
                true_positives += matrix[row][category]             
        return true_positives/(true_positives + false_positives)
        
    def calculate_false_positives(self, category, matrix):
        true_negatives = 0
        false_positives = 0
        for row in matrix.keys():
            if row != category:
                false_positives += matrix[row][category]
                
        for row in matrix.keys():
            for column in matrix.keys():
                if row == column and row != category:
                    true_negatives += matrix[row][column]
        return false_positives / (false_positives + true_negatives)
        
    def calculate_f1_score(self,category, matrix):
        precision = self.calculate_precision(category, matrix)
        recall = self.calculate_true_positives(category, matrix)
        return (2*precision*recall)/(precision+recall)
                

file = 'inputs/NoticiasArgentinas.xlsx'
categories = ['Nacional', 'Ciencia y Tecnologia', 'Deportes', 'Salud']
classifier = NewsClassifier()
classifier.split_dataset(20, file, categories)
classifier.train()
classifier.predict_title('La próxima semana habrá paro total de transporte')
roc_matrix = classifier.calculate_confusion_matrix_ROC('Nacional')
print("accuracy: ",classifier.calculate_accuracy(roc_matrix))
print("true_positives: ",classifier.calculate_true_positives('Nacional',roc_matrix))
print("precision: ",classifier.calculate_precision('Nacional', roc_matrix))
print("f1-score: ",classifier.calculate_f1_score('Nacional', roc_matrix))
print("false_positives: ",classifier.calculate_false_positives('Nacional', roc_matrix))

  (0, 57)	1
  (0, 2243)	1
  (0, 1854)	1
  (0, 4246)	1
  (0, 1352)	1
  (0, 530)	1
  (0, 2350)	1
  (0, 1987)	1
  (0, 3770)	1
  (1, 951)	1
  (1, 3734)	1
  (1, 3948)	1
  (1, 4491)	1
  (1, 651)	1
  (1, 2670)	1
  (1, 4303)	1
  (1, 1365)	1
  (1, 3890)	1
  (2, 769)	1
  (2, 646)	1
  (2, 4586)	1
  (2, 4812)	1
  (2, 5038)	1
  (2, 2957)	1
  (2, 4728)	1
  :	:
  (12324, 3443)	1
  (12324, 939)	1
  (12324, 1822)	1
  (12324, 3351)	1
  (12324, 1029)	1
  (12325, 3709)	1
  (12325, 1783)	1
  (12325, 3214)	1
  (12325, 3333)	1
  (12325, 3563)	1
  (12325, 3024)	1
  (12326, 4246)	1
  (12326, 1335)	1
  (12326, 4510)	1
  (12326, 2924)	1
  (12326, 4591)	1
  (12326, 800)	1
  (12326, 3892)	1
  (12326, 461)	1
  (12327, 1123)	1
  (12327, 1204)	1
  (12327, 4216)	1
  (12327, 1056)	1
  (12327, 3854)	1
  (12327, 2293)	1
accuracy:  0.7362958157638664
true_positives:  0.6763540290620872
precision:  0.4740740740740741
f1-score:  0.5574305933587371
false_positives:  0.24419604471195186


In [112]:
import pandas as pd

def discretize_variable(variable, umbral):
    return 1 if variable >= umbral else 0

def discretize_gre(gre, umbral = 500):
    return discretize_variable(gre, umbral)

def discretize_gpa(gpa, umbral = 3):
    return discretize_variable(gpa, umbral)

file = 'inputs/binary.csv'
students_df = pd.read_csv(file)
student_amount = len(students_df)

ranks = [1, 2, 3, 4]
admits = [0, 1]

# Creating auxiliar matrices to use
rank_df = pd.DataFrame(columns=['rank1_True', 'rank2_True', 'rank3_True', 'rank4_True'])
rank_df.loc[0] = [0, 0, 0, 0]

gre_probs = []
for rank in ranks:
    gre_probs.append([rank, 0, 0])
gre_df = pd.DataFrame(data=gre_probs, columns=['rank', 'gre_True', 'gre_False'])

gpa_probs = []
for rank in ranks:
    gpa_probs.append([rank, 0, 0])
gpa_df = pd.DataFrame(data=gpa_probs, columns=['rank', 'gpa_True', 'gpa_False'])

admit_probs = []
for rank in ranks:
    for gre in [0, 1]:
        for gpa in [0, 1]:
            admit_probs.append([rank, gre, gpa, 0, 0])
admit_df = pd.DataFrame(data=admit_probs, columns=['rank', 'gre', 'gpa', 'admit_True', 'admit_False'])

# Loading data
for student_id in students_df.index:
    rank_idx = students_df['rank'][student_id] - 1
    rank_df.loc[0][rank_idx] += 1

for student_id in students_df.index:
    gre = discretize_gre(students_df['gre'][student_id])
    gpa = discretize_gpa(students_df['gpa'][student_id])
    rank = students_df['rank'][student_id]
    gre_df.loc[gre_df['rank'] == rank , 'gre_True' if gre == 1 else 'gre_False'] += 1
    gpa_df.loc[gpa_df['rank'] == rank , 'gpa_True' if gpa == 1 else 'gpa_False'] += 1

for student_id in students_df.index:
    gpa = discretize_gpa(students_df['gpa'][student_id])
    gre = discretize_gre(students_df['gre'][student_id])
    rank = students_df['rank'][student_id]
    admit = students_df['admit'][student_id]
    admit_df.loc[(admit_df['rank'] == rank) & (admit_df['gre'] == gre) & (admit_df['gpa'] == gpa), 'admit_True' if admit == 1 else 'admit_False'] += 1

# print(rank_df)
# print(gre_df)
# print(gpa_df)
# print(admit_df)

rank_probs = []
for column in rank_df:
    rank_probs.append((rank_df[column].loc[0] + 1) / (sum(rank_df.iloc[0]) + len(rank_df.columns)))
rank_df.iloc[0] = rank_probs
print(rank_df)
print()

for i in gre_df.index:
    probs = []
    probs.append((gre_df['gre_True'][i] + 1) / (gre_df['gre_True'][i] + gre_df['gre_False'][i] + 2))
    probs.append((gre_df['gre_False'][i] + 1) / (gre_df['gre_True'][i] + gre_df['gre_False'][i] + 2))
    gre_df.at[i, 'gre_True'] = probs[0]
    gre_df.at[i, 'gre_False'] = probs[1]
print(gre_df)
print()

for i in gpa_df.index:
    probs = []
    probs.append((gpa_df['gpa_True'][i] + 1) / (gpa_df['gpa_True'][i] + gpa_df['gpa_False'][i] + 2))
    probs.append((gpa_df['gpa_False'][i] + 1) / (gpa_df['gpa_True'][i] + gpa_df['gpa_False'][i] + 2))
    gpa_df.at[i, 'gpa_True'] = probs[0]
    gpa_df.at[i, 'gpa_False'] = probs[1]
print(gpa_df)
print()

for i in admit_df.index:
    probs = []
    probs.append((admit_df['admit_True'][i] + 1) / (admit_df['admit_True'][i] + admit_df['admit_False'][i] + 2))
    probs.append((admit_df['admit_False'][i] + 1) / (admit_df['admit_True'][i] + admit_df['admit_False'][i] + 2))
    admit_df.at[i, 'admit_True'] = probs[0]
    admit_df.at[i, 'admit_False'] = probs[1]
print(admit_df)
print()

# 3.a -> P(!admitido | rango=1) = P(!admitido ^ rango=1) / P(rango=1)
# P(!admitido ^ rango=1) = Sum_GPE(0,1){Sum_GPA(0,1){P(!admitido ^ GPE ^ GPA ^ rango=1}} = P(!admitido ^ !gpe ^ !gpa ^ rango = 1) * ... * P(!admitido ^ gpe ^ gpa ^ rango = 1)
# Ej: P(!admitido ^ !gpe ^ !gpa ^ rango=1) = P(!admitido | !gpe ^ !gpa ^ rango=1) * P(!gpe | rango=1) * P(!gpa | rango=1) * P(rango=1)
# p1: P(!admitido | !gpe ^ !gpa ^ rango=1)
# p2: P(!gpe | rango=1)
# p3: P(!gpa | rango=1)
probability_3a = 0

for gre in [0, 1]:
    for gpa in [0, 1]:
        p1 = float(admit_df.loc[(admit_df['gre'] == gre) & (admit_df['gpa'] == gpa) & (admit_df['rank'] == 1), 'admit_False'].iloc[0])
        p2 = float(gre_df.loc[gre_df['rank'] == 1, 'gre_True' if gre == 1 else 'gre_False'].iloc[0])
        p3 = float(gpa_df.loc[gpa_df['rank'] == 1, 'gpa_True' if gpa == 1 else 'gpa_False'].iloc[0])

        # print(p1)
        # print(p2)
        # print(p3)
        # print()

        probability_3a += p1 * p2 * p3

print('Ejercicio 3a:', probability_3a)

# 3.b -> P(admitido | r=2 ^ !gre ^ gpa) = P(admitido ^ r=2 ^ !gre ^ gpa) / Sum_ADMIT(0,1){P(r=2 ^ !gre ^ gpa ^ ADMIT)}
# P(admitido ^ r=2 ^ !gre ^ gpa) = P(admitido | !gpe ^ gpa ^ r=2) * P(!gpe | r=2) * P(gpa | r=2) * P(r=2)
# P(!admitido ^ r=2 ^ !gre ^ gpa) = P(!admitido | !gpe ^ gpa ^ r=2) * P(!gpe | r=2) * P(gpa | r=2) * P(r=2)
# Entonces: P(admitido | r=2 ^ !gre ^ gpa) = P(admitido ^ !gpe ^ gpa ^ r=2) / (P(admitido ^ !gpe ^ gpa ^ r=2) + P(!admitido ^ !gpe ^ gpa ^ r=2))
probabilities_3b = []
for admit in [0, 1]:
    p1 = float(admit_df.loc[(admit_df['gre'] == 0) & (admit_df['gpa'] == 1) & (admit_df['rank'] == 2), 'admit_True' if admit == 1 else 'admit_False'].iloc[0])
    p2 = float(gre_df.loc[gre_df['rank'] == 2, 'gre_False'].iloc[0])
    p3 = float(gpa_df.loc[gpa_df['rank'] == 2, 'gpa_True'].iloc[0])

    # print(p1)
    # print(p2)
    # print(p3)
    # print()

    probabilities_3b.append(p1 * p2 * p3)

print('Ejercicio 3b:', probabilities_3b[1] / sum(probabilities_3b))

   rank1_True  rank2_True  rank3_True  rank4_True
0    0.153465    0.376238     0.30198    0.168317

   rank  gre_True  gre_False
0     1  0.809524   0.190476
1     2  0.810458   0.189542
2     3  0.788618   0.211382
3     4  0.782609   0.217391

   rank  gpa_True  gpa_False
0     1  0.857143   0.142857
1     2  0.823529   0.176471
2     3  0.829268   0.170732
3     4  0.797101   0.202899

    rank  gre  gpa  admit_True  admit_False
0      1    0    0    0.285714     0.714286
1      1    0    1    0.500000     0.500000
2      1    1    0    0.800000     0.200000
3      1    1    1    0.551020     0.448980
4      2    0    0    0.444444     0.555556
5      2    0    1    0.217391     0.782609
6      2    1    0    0.190476     0.809524
7      2    1    1    0.424528     0.575472
8      3    0    0    0.090909     0.909091
9      3    0    1    0.222222     0.777778
10     3    1    0    0.384615     0.615385
11     3    1    1    0.252874     0.747126
12     4    0    0    0.166667     